In [ ]:
# jupyter nbconvert --to script camera_bird_detection.ipynb

In [2]:
import tensorflow as tf
import cv2
import numpy as np
import matplotlib.pyplot as plt
import time
from djitellopy import Tello
from datetime import datetime
import os
import subprocess
from playsound import playsound
import threading

print("TensorFlow version:", tf.__version__)
print("OpenCV version:", cv2.__version__)

TensorFlow version: 2.19.0
OpenCV version: 4.11.0


In [28]:
# Constants
MODEL_PATH = "bird_detection_model_v1.keras"
SOUND_FILENAME = 'alert.wav'
CLASS_NAMES = ['no_bird', 'with_bird']
INPUT_SIZE = (180, 180)
ROWS, COLS = 8, 8  # Tile grid
THRESHOLD = 0.50
LOOP_INTERVAL = 3

In [30]:
# Load the model
model = tf.keras.models.load_model(MODEL_PATH)
print("Model loaded ✅")

Model loaded ✅


In [42]:
def crop_center_square(tile):
    h, w = tile.shape[:2]
    min_dim = min(h, w)
    y1 = (h - min_dim) // 2
    x1 = (w - min_dim) // 2
    return tile[y1:y1+min_dim, x1:x1+min_dim]

def preprocess_frame_to_tiles(frame, rows, cols, input_size):
    height, width = frame.shape[:2]
    tile_h, tile_w = height // rows, width // cols

    tiles = []
    tile_coords = []

    for r in range(rows):
        for c in range(cols):
            y1, y2 = r * tile_h, (r + 1) * tile_h
            x1, x2 = c * tile_w, (c + 1) * tile_w
            tile = frame[y1:y2, x1:x2]
            square = crop_center_square(tile)
            resized = cv2.resize(square, input_size)
            normalized = resized.astype("float32") / 255.0
            tiles.append(normalized)
            tile_coords.append((x1, y1, x2, y2))

    return np.array(tiles), tile_coords

def save_detected_bird(tile_img, cam_id, tile_id, confidence):
    now = datetime.now().strftime("%Y%m%d_%H%M%S")
    folder = "detections"
    os.makedirs(folder, exist_ok=True)
    filename = f"{folder}/cam{cam_id}_tile{tile_id}_{int(confidence*100)}_{now}.jpg"
    cv2.imwrite(filename, tile_img)
    print(f"📸 Saved detection: {filename}")

def play_sound_deterent():
    print("🔊 Playing sound...")
    sound_path = os.path.join(os.getcwd(), SOUND_FILENAME)
    return subprocess.Popen(["aplay", sound_path])

def deploy_drone():
    print("🚁 Drone: Starting mission...")

    try:
        tello = Tello()
        tello.connect()
        print("✅ Drone connected")

        time.sleep(2)

        tello.takeoff()
        print("🛫 Drone took off")

        # Move forward
        tello.move_forward(100)

        # Simulate circular path
        for _ in range(8):
            tello.move_forward(50)
            tello.rotate_clockwise(45)

        # Simulate return
        tello.rotate_clockwise(180)
        tello.move_forward(100)

        tello.land()
        print("✅ Drone landed successfully")

    except Exception as e:
        print(f"⚠️ Drone error: {str(e)}")
        print("❌ Flight mission aborted, continuing system...")

        # Optional safety landing if drone took off but failed mid-flight
        # try:
        #     tello.land()
        # except:
        #     pass

def respond_to_bird():
    sound_proc = play_sound_deterent()
    deploy_drone()
    sound_proc.terminate()  # stop the sound (optional)

In [44]:
# Camera setup
capture_one = cv2.VideoCapture(0)
capture_two = cv2.VideoCapture(1)

capture_one.set(cv2.CAP_PROP_FRAME_WIDTH, 1920)
capture_one.set(cv2.CAP_PROP_FRAME_HEIGHT, 1080)
capture_two.set(cv2.CAP_PROP_FRAME_WIDTH, 1920)
capture_two.set(cv2.CAP_PROP_FRAME_HEIGHT, 1080)

loop_count = 0

while True:
    for cam_id, cap in enumerate([capture_one, capture_two]):
        ret, frame = cap.read()
        if not ret:
            print(f"Failed to read from camera {cam_id}")
            continue

        tiles, tile_coords = preprocess_frame_to_tiles(frame, ROWS, COLS, INPUT_SIZE)

        input_batch = np.array(tiles)
        preds = model.predict(input_batch)

        for i, ((x1, y1, x2, y2), pred) in enumerate(zip(tile_coords, preds)):
            pred_class = np.argmax(pred)
            confidence = pred[pred_class]
            label = CLASS_NAMES[pred_class]
            print(f"[CAM {cam_id}] Tile {i} [{x1},{y1}] → {label} ({confidence:.2f})")

            if label == "with_bird" and confidence >= THRESHOLD:
                print("Bird Detected...")
                respond_to_bird()
                save_detected_bird(tiles[i], cam_id, i, confidence)
                break

    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

    print("sleepp...")
    time.sleep(LOOP_INTERVAL)
    loop_count = loop_count + 1

    if loop_count >= 3:
        print("break the loop...")
        break

capture_one.release()
capture_two.release()
cv2.destroyAllWindows()


Failed to read from camera 0
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
[CAM 1] Tile 0 [0,0] → with_bird (0.51)
Bird Detected...
🔊 Playing sound...


FileNotFoundError: [WinError 2] The system cannot find the file specified